In [1]:
###

# Deep Learning (RNN) Demo for Load Forecasting

## SEC 1: Import all the packages needed

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.python.ops import seq2seq
from tensorflow.python.ops import rnn_cell
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
import random as rd
import argparse
import os, sys
import csv
import math
import time
import matplotlib.pyplot as pl

## SEC 2: Load demand data from excel file

define class for dataset

In [3]:
class County:
    def __init__(self,parsename):
        self.parsename = parsename
        dataframe = xls.parse(parsename)
        self.date = dataframe['Date']
        self.hour = dataframe['Hr_End']
        self.demand = dataframe['RT_Demand']
        self.drybulb = dataframe['Dry_Bulb']
        self.dewpnt = dataframe['Dew_Point']
    def disp_all(self):
        print self.dataframe
    def get_all(self):
        return self.dataframe

load data

In [4]:
xls = pd.ExcelFile('smd_hourly.xls')
INC = County('ISO NE CA')
ME = County('ME')
NH = County('NH')
VT = County('VT')
CT = County('CT')
RI = County('RI')
SEMA = County('SEMA')
WCMA = County('WCMA')
NEMA = County('NEMA')

## SEC 3: setting all global parameters

In [5]:
data_dir = './data/' # directory contains input data
num_epoches = 500# training epoches for each customer samples
input_seq_size = 7*24 # input size
test_batch_size = 1 # days of a batch
valid_batch_size = 14
train_batch_size = 1
data_dim = 1 # same time of a week
output_seq_size = 24
totalen = np.array(INC.demand).shape[0]/output_seq_size
n_hidden = 1 # input size
num_layers = 3

## SEC 4: split dataset into training, cross-validation, and test

concatenate data into database

In [6]:
# DEMAND MATRIX 9 X LENGTH, 9: INC is total, index with 0, other substations are from 1 -> 8
tmp = np.array(INC.demand, dtype = np.float32)
demand_mat = tmp.reshape([tmp.shape[0]/output_seq_size,output_seq_size,1])
demand_mat = demand_mat/25000
#demand_mat = np.concatenate([demand_mat,np.array(ME.demand).reshape([1,np.array(ME.demand).shape[0],1])],axis = 0)
#demand_mat = np.concatenate([demand_mat,np.array(NH.demand).reshape([1,np.array(NH.demand).shape[0],1])],axis = 0)
#demand_mat = np.concatenate([demand_mat,np.array(VT.demand).reshape([1,np.array(VT.demand).shape[0],1])],axis = 0)
#demand_mat = np.concatenate([demand_mat,np.array(CT.demand).reshape([1,np.array(CT.demand).shape[0],1])],axis = 0)
#demand_mat = np.concatenate([demand_mat,np.array(RI.demand).reshape([1,np.array(RI.demand).shape[0],1])],axis = 0)
#demand_mat = np.concatenate([demand_mat,np.array(SEMA.demand).reshape([1,np.array(SEMA.demand).shape[0],1])],axis = 0)
#demand_mat = np.concatenate([demand_mat,np.array(WCMA.demand).reshape([1,np.array(WCMA.demand).shape[0],1])],axis = 0)
#demand_mat = np.concatenate([demand_mat,np.array(NEMA.demand).reshape([1,np.array(NEMA.demand).shape[0],1])],axis = 0)
print demand_mat.shape
# DRY BULB MATRIX 9 X LENGTH, 9: INC is total, index with 0, other substations are from 1 -> 8
tmp = np.array(INC.drybulb, dtype = np.float32)
drybulb_mat = tmp.reshape([tmp.shape[0]/output_seq_size,output_seq_size,1])
drybulb_mat = drybulb_mat/100
#drybulb_mat = np.concatenate([drybulb_mat,np.array(ME.drybulb).reshape([1,np.array(ME.drybulb).shape[0],1])],axis = 0)
#drybulb_mat = np.concatenate([drybulb_mat,np.array(NH.drybulb).reshape([1,np.array(NH.drybulb).shape[0],1])],axis = 0)
#drybulb_mat = np.concatenate([drybulb_mat,np.array(VT.drybulb).reshape([1,np.array(VT.drybulb).shape[0],1])],axis = 0)
#drybulb_mat = np.concatenate([drybulb_mat,np.array(CT.drybulb).reshape([1,np.array(CT.drybulb).shape[0],1])],axis = 0)
#drybulb_mat = np.concatenate([drybulb_mat,np.array(RI.drybulb).reshape([1,np.array(RI.drybulb).shape[0],1])],axis = 0)
#drybulb_mat = np.concatenate([drybulb_mat,np.array(SEMA.drybulb).reshape([1,np.array(SEMA.drybulb).shape[0],1])],axis = 0)
#drybulb_mat = np.concatenate([drybulb_mat,np.array(WCMA.drybulb).reshape([1,np.array(WCMA.drybulb).shape[0],1])],axis = 0)
#drybulb_mat = np.concatenate([drybulb_mat,np.array(NEMA.drybulb).reshape([1,np.array(NEMA.drybulb).shape[0],1])],axis = 0)
#print drybulb_mat.shape
# DEW PNT MATRIX 9 X LENGTH, 9: INC is total, index with 0, other substations are from 1 -> 8
tmp = np.array(INC.dewpnt, dtype = np.float32)
dewpnt_mat = tmp.reshape([tmp.shape[0]/output_seq_size,output_seq_size,1])
dewpnt_mat = dewpnt_mat/100
#dewpnt_mat = np.concatenate([dewpnt_mat,np.array(ME.dewpnt).reshape([1,np.array(ME.dewpnt).shape[0],1])],axis = 0)
#dewpnt_mat = np.concatenate([dewpnt_mat,np.array(NH.dewpnt).reshape([1,np.array(NH.dewpnt).shape[0],1])],axis = 0)
#dewpnt_mat = np.concatenate([dewpnt_mat,np.array(VT.dewpnt).reshape([1,np.array(VT.dewpnt).shape[0],1])],axis = 0)
#dewpnt_mat = np.concatenate([dewpnt_mat,np.array(CT.dewpnt).reshape([1,np.array(CT.dewpnt).shape[0],1])],axis = 0)
#dewpnt_mat = np.concatenate([dewpnt_mat,np.array(RI.dewpnt).reshape([1,np.array(RI.dewpnt).shape[0],1])],axis = 0)
#dewpnt_mat = np.concatenate([dewpnt_mat,np.array(SEMA.dewpnt).reshape([1,np.array(SEMA.dewpnt).shape[0],1])],axis = 0)
#dewpnt_mat = np.concatenate([dewpnt_mat,np.array(WCMA.dewpnt).reshape([1,np.array(WCMA.dewpnt).shape[0],1])],axis = 0)
#dewpnt_mat = np.concatenate([dewpnt_mat,np.array(NEMA.dewpnt).reshape([1,np.array(NEMA.dewpnt).shape[0],1])],axis = 0)
#print dewpnt_mat.shape

#db = np.concatenate([demand_mat,dewpnt_mat,drybulb_mat],axis = 2)
#db = np.concatenate([demand_mat,dewpnt_mat],axis = 2)
db = demand_mat

print db.shape
print db

(244, 24, 1)
(244, 24, 1)
[[[ 0.46049318]
  [ 0.43851206]
  [ 0.4229148 ]
  ..., 
  [ 0.55868357]
  [ 0.52327406]
  [ 0.48644769]]

 [[ 0.45718294]
  [ 0.44048843]
  [ 0.43136075]
  ..., 
  [ 0.57150865]
  [ 0.53509986]
  [ 0.49743447]]

 [[ 0.46581072]
  [ 0.4456512 ]
  [ 0.43486208]
  ..., 
  [ 0.56737697]
  [ 0.5218094 ]
  [ 0.48097339]]

 ..., 
 [[ 0.52502024]
  [ 0.50013936]
  [ 0.48680067]
  ..., 
  [ 0.6965189 ]
  [ 0.6219433 ]
  [ 0.55489725]]

 [[ 0.50779676]
  [ 0.47691453]
  [ 0.45675594]
  ..., 
  [ 0.67518634]
  [ 0.61018306]
  [ 0.55081195]]

 [[ 0.50914031]
  [ 0.48262376]
  [ 0.46657395]
  ..., 
  [ 0.71185482]
  [ 0.64936733]
  [ 0.59077376]]]


split into 3 parts using part array

In [7]:
#define id arrays
test_id = np.array(test_batch_size)
valid_id = np.array(valid_batch_size)
train_id = np.array(totalen-test_batch_size-valid_batch_size-input_seq_size)

#give values to id arrays
rang = range(input_seq_size/output_seq_size,totalen-test_batch_size)
valid_id = rd.sample(rang,valid_batch_size)
test_id = np.array(range(totalen-test_batch_size,totalen))
train_id = set(range(input_seq_size/output_seq_size,totalen-test_batch_size))-set(valid_id)

#sort three id array
valid_id = np.sort(valid_id)
test_id = np.sort(test_id)
train_id = np.array(list(train_id))
print valid_id
print test_id
print train_id

[ 13  38  68  74  99 112 120 142 154 165 183 207 208 215]
[243]
[  7   8   9  10  11  12  14  15  16  17  18  19  20  21  22  23  24  25
  26  27  28  29  30  31  32  33  34  35  36  37  39  40  41  42  43  44
  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62
  63  64  65  66  67  69  70  71  72  73  75  76  77  78  79  80  81  82
  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98 100 101
 102 103 104 105 106 107 108 109 110 111 113 114 115 116 117 118 119 121
 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139
 140 141 143 144 145 146 147 148 149 150 151 152 153 155 156 157 158 159
 160 161 162 163 164 166 167 168 169 170 171 172 173 174 175 176 177 178
 179 180 181 182 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 209 210 211 212 213 214 216 217 218
 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236
 237 238 239 240 241 242]


### Step 4: define data generating function code. 
which generate a batch of batch-size large sequence data. the data is feature_size dims width and is a time series of float32 of steps steps. inputs and outputs are:

inputs:
----n_batch: number of samples in a batch
----steps: the sequence length of a sample data
----feature_size: dimensions of a single time step data frame

outputs:
----X inputs, shape(n_batch,steps,feature_size)
----Y outputs should be, shape(n_batch,)

In [8]:
def train_data_gen():
    X = np.zeros((input_seq_size,train_batch_size,data_dim))
    Y = np.zeros((output_seq_size,train_batch_size,data_dim))
    count = 0
    rang = range(input_seq_size/output_seq_size,train_id.shape[0])
    train_rd = rd.sample(rang,train_batch_size)
    train_rd = np.sort(train_rd)
    for i in train_rd:
        Y[:,count,:] = db[i,:,:]
        X[:,count,:] = (db[i-input_seq_size/output_seq_size:i,:,:]).reshape([input_seq_size,data_dim])
        count = count + 1
    return (X,Y)

In [9]:
def valid_data_gen():
    X = np.zeros((input_seq_size,train_batch_size,data_dim))
    Y = np.zeros((output_seq_size,train_batch_size,data_dim))
    count = 0
    rang = range(input_seq_size/output_seq_size,valid_id.shape[0])
    valid_rd = rd.sample(rang,train_batch_size)
    valid_rd = np.sort(valid_rd)
    for i in valid_rd:
        Y[:,count,:] = db[i,:,:]
        X[:,count,:] = (db[i-input_seq_size/output_seq_size:i,:,:]).reshape([input_seq_size,data_dim])
        count = count + 1
    return (X,Y)

In [10]:
def test_data_gen():
    X = np.zeros((input_seq_size,test_batch_size,data_dim))
    Y = np.zeros((output_seq_size,test_batch_size,data_dim))
    count = 0
    for i in test_id:
        Y[:,count,:] = db[i,:,:]
        X[:,count,:] = (db[i-input_seq_size/output_seq_size:i,:,:]).reshape([input_seq_size,data_dim])
        count = count + 1
    return (X,Y)

In [11]:
# code testing
#"""
(x,y) = valid_data_gen()
print x.shape
print y.shape
#"""
"""
count = 0
X = np.zeros((test_batch_size,n_steps,feature_size))
Y = np.zeros((test_batch_size,feature_size))
for i in test_id:
    print i
    Y[0] = db[:,i,:]
    X[0] = db[:,i-n_steps:i,:]
    count = count + 1
    if count == 3:
        break
print Y
print Y.shape
print X
print X.shape
"""
"""
X = np.zeros((train_batch_size,n_steps,feature_size))
Y = np.zeros((train_batch_size,feature_size))
count = 0
rang = range(n_steps,train_id.shape[0])
train_rd = rd.sample(rang,train_batch_size)
train_rd = np.sort(train_rd)
for i in train_id:
    Y[count] = db[:,i,:]
    X[count] = db[:,i-n_steps:i,:]
    count = count + 1
    if count == 3:
        break
print Y
print Y.shape
print X
print X.shape
"""

(168, 1, 1)
(24, 1, 1)


'\nX = np.zeros((train_batch_size,n_steps,feature_size))\nY = np.zeros((train_batch_size,feature_size))\ncount = 0\nrang = range(n_steps,train_id.shape[0])\ntrain_rd = rd.sample(rang,train_batch_size)\ntrain_rd = np.sort(train_rd)\nfor i in train_id:\n    Y[count] = db[:,i,:]\n    X[count] = db[:,i-n_steps:i,:]\n    count = count + 1\n    if count == 3:\n        break\nprint Y\nprint Y.shape\nprint X\nprint X.shape\n'

In [12]:
def variable_summaries(var, name):
    """Attach a lot of summaries to a Tensor."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.scalar_summary('mean/' + name, mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
        tf.scalar_summary('sttdev/' + name, stddev)
        tf.scalar_summary('max/' + name, tf.reduce_max(var))
        tf.scalar_summary('min/' + name, tf.reduce_min(var))
        tf.histogram_summary(name, var)

### Step 5: construct RNN model

In [13]:
_X = [tf.placeholder(tf.float32, shape = [train_batch_size, data_dim]) for _ in xrange(input_seq_size)]
_Y = [tf.placeholder(tf.float32, shape = [train_batch_size, data_dim]) for _ in xrange(output_seq_size)]
#_X = tf.reshape(_X, [-1, data_dim])
#_Y = tf.reshape(_Y, [-1, data_dim])
weights_in = [ tf.Variable(tf.random_normal([data_dim, n_hidden])) for _ in xrange(input_seq_size) ]
bias_in = [ tf.Variable(tf.random_normal([n_hidden])) for _ in xrange(input_seq_size) ]

weights_out = [ tf.Variable(tf.random_normal([n_hidden, data_dim])) for _ in xrange(output_seq_size)]
bias_out = [ tf.Variable(tf.random_normal([data_dim])) for _ in xrange(output_seq_size)]

print _X[0]
encoder_inputs = [ tf.matmul(_X[i], weights_in[i]) + bias_in[i] for i in xrange(input_seq_size) ]
decoder_inputs = [ tf.matmul(_Y[i], weights_in[i]) + bias_in[i] for i in xrange(output_seq_size) ]




"""
#with tf.device('/gpu:0'):
_X = tf.placeholder("float",[None,input_seq_size,data_dim])
_Y = tf.placeholder("float",[None,output_seq_size,data_dim])

# Define weights
weights = {
    'in': tf.Variable(tf.random_normal([data_dim, n_hidden])), # Hidden layer weights
    'out': tf.Variable(tf.random_normal([data_dim, n_hidden]))
}
biases = {
    'in': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_hidden]))
}
# input shape: (batch_size, n_steps, n_input)
_X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
_Y = tf.transpose(_Y, [1, 0, 2])
# Reshape to prepare input to hidden activation

_X = tf.reshape(_X, [-1, data_dim]) # (n_steps*batch_size, n_input)
_Y = tf.reshape(_Y, [-1, data_dim]) # (n_steps*batch_size, n_input)

#_X = tf.matmul(_X, weights['in']) + biases['in']
#_Y = tf.matmul(_Y, weights['out']) + biases['out']

_X = tf.split(0, input_seq_size, _X)
_Y = tf.split(0, output_seq_size, _Y)
"""

cell = rnn_cell.GRUCell(n_hidden)
dropout = tf.constant(0.75, dtype = tf.float32)
cell = rnn_cell.DropoutWrapper(cell, output_keep_prob = dropout)
cell = rnn_cell.MultiRNNCell([cell]*num_layers)

model_outputs, states = seq2seq.basic_rnn_seq2seq(_X, _Y, cell)

_pred = [ tf.matmul(model_outputs[i], weights_out[i]) + bias_out[i] for i in xrange(output_seq_size)]

reshaped_outputs = tf.reshape(_pred, [-1])
reshaped_results = tf.reshape(_Y, [-1])

cost = tf.reduce_mean(tf.pow(reshaped_outputs-reshaped_results,2))

variable_summaries(cost, 'cost')
#compute parameter updates
#optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(cost)
optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

Tensor("Placeholder:0", shape=(1, 1), dtype=float32)


In [40]:
def maxe(predictions, targets):
    return np.max(abs(predictions-targets))

In [15]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [16]:
def mape(predictions, targets):
    return np.mean(abs(predictions-targets)/targets)

In [17]:
outlist = np.zeros([(num_epoches/10),output_seq_size*test_batch_size])
kind = 0
time1 = time.time()
# generate test data
test_x,test_y = test_data_gen()
test_y_unknown = np.zeros((output_seq_size,test_batch_size,data_dim)) # test if it can use for prediction
test_x = test_x.astype(np.float32)
test_y = test_y.astype(np.float32)
tex_list = {key: value for (key, value) in zip(_X, test_x)}
tey_list = {key: value for (key, value) in zip(_Y, test_y_unknown)}
### Execute
# Initializing the variables
init = tf.initialize_all_variables()

In [18]:
"""
def pred_tomorrow(dx, dy, dstate):
    pred_y = np.zeros((test_batch_size/24,24,feature_size))
    Xupdate = np.zeros((test_batch_size/24,1,feature_size))
    for timeslot in range(24):
        Xmat = np.zeros((test_batch_size/24,n_steps,feature_size))
        Ymat = np.zeros((test_batch_size/24,feature_size))
        tmpXmat = np.zeros((1,n_steps,feature_size))
        tmpYmat = np.zeros((1,feature_size))
        count = 0
        for row in (r for r in range(test_batch_size) if np.mod(r,24)==timeslot):
            tmpXmat = dx[row,:,:]
            tmpYmat = dy[row,:]
            Xmat[count,:,:] = tmpXmat
            Ymat[count,:] = tmpYmat
            count = count + 1
        #print Xmat
        if timeslot > 0:
            #print 'Xupdate'
            #print Xupdate.reshape((test_batch_size/24,1))
            #print 'Xmat'
            #print Xmat[:,-1,:]
            Xmat[:,-1,:] = Xupdate.reshape((test_batch_size/24,1))
            #print 'Xmat new'
            #print Xmat[:,-1,:]
        #print Ymat
        #print Xmat
        output_tmp_ex = sess.run(pred,feed_dict = {x:Xmat,y:Ymat,istate:dstate})
        tmpout = output_tmp_ex[:,0]
        #print tmpout
        Xupdate = tmpout.reshape((test_batch_size/24,1,feature_size))
        #print Xupdate
        pred_y[:,timeslot,:] = Xupdate.reshape((test_batch_size/24,1))
    pred_y = pred_y.reshape((test_batch_size,1))
    return pred_y
"""

"\ndef pred_tomorrow(dx, dy, dstate):\n    pred_y = np.zeros((test_batch_size/24,24,feature_size))\n    Xupdate = np.zeros((test_batch_size/24,1,feature_size))\n    for timeslot in range(24):\n        Xmat = np.zeros((test_batch_size/24,n_steps,feature_size))\n        Ymat = np.zeros((test_batch_size/24,feature_size))\n        tmpXmat = np.zeros((1,n_steps,feature_size))\n        tmpYmat = np.zeros((1,feature_size))\n        count = 0\n        for row in (r for r in range(test_batch_size) if np.mod(r,24)==timeslot):\n            tmpXmat = dx[row,:,:]\n            tmpYmat = dy[row,:]\n            Xmat[count,:,:] = tmpXmat\n            Ymat[count,:] = tmpYmat\n            count = count + 1\n        #print Xmat\n        if timeslot > 0:\n            #print 'Xupdate'\n            #print Xupdate.reshape((test_batch_size/24,1))\n            #print 'Xmat'\n            #print Xmat[:,-1,:]\n            Xmat[:,-1,:] = Xupdate.reshape((test_batch_size/24,1))\n            #print 'Xmat new'\n      

In [19]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    # Create a summary to monitor cost function
    #tf.scalar_summary("loss", cost)
    # Merge all summaries to a single operator
    merged_summary_op = tf.merge_all_summaries()

    # tensorboard info.# Set logs writer into folder /tmp/tensorflow_logs
    summary_writer = tf.train.SummaryWriter('/tmp/tensorflow_logs', graph_def=sess.graph)

    #initialize all variables in the model
    sess.run(init)
    costs = []
    costs_test = []
    for k in range(num_epoches):
        #Generate Data for each epoch
        #What this does is it creates a list of of elements of length seq_len, each of size [batch_size,input_size]
        #this is required to feed data into rnn.rnn
        #print traindays
        X,Y = train_data_gen()
        X = X.astype(np.float32)
        Y = Y.astype(np.float32)
        #Create the dictionary of inputs to feed into sess.run
        #if k < 0:
        #    sess.run(optimizer2,feed_dict={x:X,y:Y,istate:np.zeros((train_batch_size,num_layers*2*n_hidden))})
        #else:
        x_list = {key: value for (key, value) in zip(_X, X)}
        y_list = {key: value for (key, value) in zip(_Y, Y)}
        summary, err, _ = sess.run([merged_summary_op, cost, optimizer], feed_dict=dict(x_list.items() + y_list.items()))
        costs.append(err)
        #perform an update on the parameters
        #cost1 = sess.run(cost,feed_dict = {x:test_x,y:test_y,istate:np.zeros((test_batch_size,num_layers*2*n_hidden))} )
        if k%10==0:
            print "Iter " + str(k) + ", Minibatch Loss ---- Train = " + str(err)
            err_test,pred_test = sess.run([cost, reshaped_outputs], feed_dict=dict(tex_list.items() + tey_list.items()))
            costs_test.append(err_test)
            outlist[kind,:] = pred_test.copy().T
            kind = kind + 1

Iter 0, Minibatch Loss ---- Train = 1.20618
Iter 10, Minibatch Loss ---- Train = 1.32402
Iter 20, Minibatch Loss ---- Train = 1.22759
Iter 30, Minibatch Loss ---- Train = 1.13696
Iter 40, Minibatch Loss ---- Train = 1.12893
Iter 50, Minibatch Loss ---- Train = 1.263
Iter 60, Minibatch Loss ---- Train = 1.22584
Iter 70, Minibatch Loss ---- Train = 1.06313
Iter 80, Minibatch Loss ---- Train = 1.37551
Iter 90, Minibatch Loss ---- Train = 1.00327
Iter 100, Minibatch Loss ---- Train = 0.988994
Iter 110, Minibatch Loss ---- Train = 1.00869
Iter 120, Minibatch Loss ---- Train = 0.989676
Iter 130, Minibatch Loss ---- Train = 0.988666
Iter 140, Minibatch Loss ---- Train = 0.975553
Iter 150, Minibatch Loss ---- Train = 1.02046
Iter 160, Minibatch Loss ---- Train = 0.933824
Iter 170, Minibatch Loss ---- Train = 0.85038
Iter 180, Minibatch Loss ---- Train = 0.837052
Iter 190, Minibatch Loss ---- Train = 0.840933
Iter 200, Minibatch Loss ---- Train = 0.939862
Iter 210, Minibatch Loss ---- Train = 0

In [20]:
time2 = time.time()
print time2-time1
costlist = np.array(costs)
costlist_te = np.array(costs_test)
prefix = './seq2seq/'
DataFrame(costlist).to_csv(prefix + 'costfile_train.csv')
DataFrame(costlist_te).to_csv(prefix + 'costfile_test.csv')
outlist = outlist * 25000
actuaload = np.reshape(test_y, [-1])
actuaload = (actuaload*25000).T
prediction = np.array(outlist[-1,:])
DataFrame(actuaload).to_csv(prefix + 'actual_load.csv')
DataFrame(prediction).to_csv(prefix + 'prediction.csv')

162.106878996


In [42]:
RList = np.zeros([(num_epoches/10)])
rmseList = np.zeros([(num_epoches/10)])
maxeList = np.zeros([(num_epoches/10)])
mapeList = np.zeros([(num_epoches/10)])
actuaload = actuaload.reshape((1,test_batch_size*output_seq_size))
for i in range(kind):
    out = np.array(outlist[i])
    tmp = out.T.reshape((1,test_batch_size*output_seq_size))
    RList[i] = np.corrcoef(tmp,actuaload)[0,1]
    rmseList[i] = rmse(tmp,actuaload)
    maxeList[i] = maxe(tmp,actuaload)
    mapeList[i] = mape(tmp,actuaload)
DataFrame(RList).to_csv(prefix + 'R.csv')
DataFrame(rmseList).to_csv(prefix + 'RMSE.csv')
DataFrame(maxeList).to_csv(prefix + 'MAXE.csv')
DataFrame(mapeList).to_csv(prefix + 'MAPE.csv')

In [43]:
plt.plot(costs)
plt.show()